In [1]:
import folium
import folium.plugins as plugins
import pandas as pd
import json
import requests
import time
from folium.plugins import HeatMapWithTime
import datetime

In [2]:
with open('Binhtan_f0.json', 'r') as j:
    json_data = json.load(j)
    # print(json_data)

In [4]:
#input how long you wish to see
mins = input('How many mins do you want to see? ')
number_of_times = (int(mins)*60)/30

In [5]:
#this is how many times the loop will pull data from the API
number_of_times

6.0

In [6]:
#the loop will pull live data every 30 secs and append it into a dataframe
cumulative = pd.DataFrame()
startTime = time.time()
for i in range(int(number_of_times)):
    url = ("https://api.data.gov.sg/v1/transport/taxi-availability")
    response = requests.get(url)
    data = response.json()
    df = pd.io.json.json_normalize(data['features'])
    coordinateslist = df['geometry.coordinates'].tolist()
    df1 = pd.DataFrame(coordinateslist)
    result = df1.transpose()
    result.columns = ['coordinates']
    result['Timestamp'] = (df['properties.timestamp'][0])
    cumulative = cumulative.append(result) 
    time.sleep(30)
endTime = time.time()
elapsedTime = endTime - startTime
print("Elapsed Time = %s" % elapsedTime,'seconds')

C:\Users\MYPC~1\AppData\Local\Temp/ipykernel_17512/401880637.py:8: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  df = pd.io.json.json_normalize(data['features'])


Elapsed Time = 183.06126427650452 seconds


In [7]:
cumulative

,coordinates,Timestamp
0,"[103.62587, 1.29]",2021-11-16T15:00:39+08:00
1,"[103.62837, 1.31351]",2021-11-16T15:00:39+08:00
2,"[103.629376516667, 1.28008191666667]",2021-11-16T15:00:39+08:00
3,"[103.68106, 1.34]",2021-11-16T15:00:39+08:00
4,"[103.68283, 1.32293]",2021-11-16T15:00:39+08:00
...,...,...
1430,"[103.98733, 1.35492]",2021-11-16T15:10:40+08:00
1431,"[103.98814, 1.36]",2021-11-16T15:10:40+08:00
1432,"[103.9888214, 1.36066706666667]",2021-11-16T15:10:40+08:00
1433,"[103.988822333333, 1.360977]",2021-11-16T15:10:40+08:00


In [9]:
#the data has to be formatted into a pandas dataframe from geojson format for easy manipulation
cumulative['coordinates'] = cumulative['coordinates'].astype(str)
Latitude = []
Longitude = []
coordinates = []
for i in cumulative['coordinates']:
    i = i.split(", ")
    lat = i[1][:-1]
    long = i[0][1:]
    Latitude.append(lat)
    Longitude.append(long)
cumulative['Latitude'] = Latitude
cumulative['Longitude'] = Longitude

In [10]:
cumulative

,coordinates,Timestamp,Latitude,Longitude
0,"[103.6145315, 1.24791766666667]",2021-11-20T16:55:04+08:00,1.24791766666667,103.6145315
1,"[103.6147, 1.24747]",2021-11-20T16:55:04+08:00,1.24747,103.6147
2,"[103.64645, 1.31681]",2021-11-20T16:55:04+08:00,1.31681,103.64645
3,"[103.650201016667, 1.33379711666667]",2021-11-20T16:55:04+08:00,1.33379711666667,103.650201016667
4,"[103.6599, 1.30941]",2021-11-20T16:55:04+08:00,1.30941,103.6599
...,...,...,...,...
2324,"[103.989749666667, 1.36117783333333]",2021-11-20T16:59:34+08:00,1.36117783333333,103.989749666667
2325,"[103.98988, 1.36092]",2021-11-20T16:59:34+08:00,1.36092,103.98988
2326,"[103.99, 1.36111]",2021-11-20T16:59:34+08:00,1.36111,103.99
2327,"[103.99256, 1.39026]",2021-11-20T16:59:34+08:00,1.39026,103.99256


In [ ]:
cumulative['Timestamp']

In [14]:
#converted Timestamp into datetime format and using it as an index
cumulative['Timestamp']= pd.to_datetime(cumulative['Timestamp'])
time_index = []
for i in cumulative['Timestamp'].unique():
    time_index.append(i)
date_strings = [dt.strftime('%d/%m/%Y, %H:%M:%S') for dt in time_index]

    

In [15]:
date_strings

['20/11/2021, 16:55:04',
 '20/11/2021, 16:56:04',
 '20/11/2021, 16:57:04',
 '20/11/2021, 16:57:34',
 '20/11/2021, 16:58:04',
 '20/11/2021, 16:58:34',
 '20/11/2021, 16:59:34']

In [12]:
#formatting the data into a nested list 
lat_long_list = []
for i in cumulative['Timestamp'].unique():
    temp=[]
    for index, instance in cumulative[cumulative['Timestamp'] == i].iterrows():
        temp.append([instance['Latitude'],instance['Longitude']])
    lat_long_list.append(temp)
    

In [13]:
lat_long_list


[[['1.29', '103.62587'],
  ['1.31351', '103.62837'],
  ['1.28008191666667', '103.629376516667'],
  ['1.34', '103.68106'],
  ['1.32293', '103.68283'],
  ['1.341088', '103.6856'],
  ['1.34094768333333', '103.685657766667'],
  ['1.34', '103.68764'],
  ['1.34', '103.68878'],
  ['1.34126', '103.69015'],
  ['1.34', '103.69076'],
  ['1.35', '103.69278'],
  ['1.34', '103.69367'],
  ['1.33753', '103.69425'],
  ['1.34498', '103.69566'],
  ['1.34997638333333', '103.696332866667'],
  ['1.34006716666667', '103.6969895'],
  ['1.34888', '103.69714'],
  ['1.39071', '103.6984'],
  ['1.34', '103.69891'],
  ['1.34754816666667', '103.699405333333'],
  ['1.349853', '103.7004'],
  ['1.337602', '103.703029666667'],
  ['1.33891', '103.70429'],
  ['1.34324', '103.70529'],
  ['1.34471766666667', '103.705393833333'],
  ['1.3431', '103.70576'],
  ['1.34531183333333', '103.7061835'],
  ['1.340665', '103.7064'],
  ['1.34072', '103.70659'],
  ['1.34', '103.70695'],
  ['1.34', '103.70744'],
  ['1.34476', '103.70822']

In [10]:
#Choosing the map type 
m = folium.Map(location=[1.352083,103.819839],zoom_start = 11, tiles="https://tiles.stadiamaps.com/tiles/alidade_smooth_dark/{z}/{x}/{y}{r}.png",attr="Stadia.AlidadeSmoothDark") 

#Plot it on the map
HeatMapWithTime(lat_long_list,radius=5,auto_play=True,position='bottomright',name="cluster",index=date_strings,max_opacity=0.7).add_to(m)

# Display the map
m


In [12]:
#saving it as a HTML file 
m.save('a.html')